## The Battle of Neighborhoods Introduction and Business Problem

### Introduction
The purpose of this capstone project is to help people explore facilities around their neighborhood which in turn will help them make smart and efficient decision on selecting good neighborhoods in Toronto. Tons of research is needed to be done for good housing prices and other such assets. Ease of access to malls, supermarkets, pharmacies, schools etc are a few factors that people look at when considering a neighborhood and this project helps in that.
Median housing price and better school according to ratings, crime rates of that particular area, road connectivity, weather conditions, good management for emergency etc are also analyzed in this project. 

### Foursquare API
This project will leverage Four-square API as a data gathering source as FourSquare has a database of millions of places, especially their places API which provides the ability to perform location search, location sharing and details about a business.

### Clustering Approach
To compare the similarities between different cities in Canada, neighborhoods will be segmented using clustering algorithms. Specifically, k means algorithm will be used

### Problem which we are trying to solve:
The purpose of this project is to suggest better neighborhoods in a new city for people who are just about to move in. Ease of access to malls, supermarkets, pharmacies, schools etc are a few factors that people look at when considering a neighborhood and this project helps in that. Median housing price and better school according to ratings, crime rates of that particular area, road connectivity, weather conditions, good management for emergency etc are also analyzed in this project.



### Data 
Data source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. The information obtained per venue as follows:

1. Neighborhood
2. Neighborhood Latitude
3. Neighborhood Longitude
4. Venue
5. Name of the venue e.g. the name of a store or restaurant
6. Venue Latitude
7. Venue Longitude
8. Venue Category

In [1]:
!pip install geocoder
!pip install folium

     |████████████████████████████████| 98 kB 4.1 MB/s eta 0:00:011
     |████████████████████████████████| 93 kB 3.5 MB/s  eta 0:00:01


### Importing all imp libraries

In [2]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Some data extraction, converting to DF and cleaning

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extract_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')

In [5]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [7]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [8]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [10]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [12]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df_2.describe()

,Postalcode,Borough,Neighborhood
count,180,180,180
unique,180,11,100
top,M3Y\n,Not assigned\n,Not assigned\n
freq,1,77,77


In [13]:
print(df_2.shape)

(180, 3)
